## Анализ воронок пользователей

###  Функции для интеграции с ClickHouse

В файле `some_funcs` есть класс `simple_ch_client` c 3 функциями
* get_clickhouse_data
* get_clickhouse_df
* upload

Сначала надо создать экземпляр класса, инициализировав его начальными параметрами - хост, пользователь, пароль и путь к сертификату

In [266]:
import some_funcs
from some_funcs import simple_ch_client

In [265]:
CH_HOST = 'https://rc1a-rt82aoo61ovjyuxf.mdb.yandexcloud.net:8443'
CH_USER = 'z_sergey'

with open('../.chpass.txt') as f:
    CH_PASS = f.read().strip()

cacert = 'YandexInternalRootCA.crt'

/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py:818: UserWarning: The following variables cannot be serialized: f
  warnings.warn(message)


In [267]:
my_client = simple_ch_client(CH_HOST, CH_USER, CH_PASS, cacert)

### Разминка: смотрим на пути пользователей

In [264]:
q = '''
SELECT
    URL,
    uniq(ClientID) as users,
    count() as hits
FROM metrica_data.hits
GROUP BY URL
ORDER BY users DESC
LIMIT 10
FORMAT TabSeparatedWithNames
'''

my_client.get_clickhouse_df(q)

,URL,users,hits
0,https://supermarket.ru/,40191,167248
1,https://supermarket.ru/search,31297,376060
2,https://supermarket.ru/promo_actions,22684,93260
3,https://supermarket.ru/product_1174,20912,37002
4,https://supermarket.ru/catalog,13424,60112
5,https://supermarket.ru/basket,13324,196363
6,https://supermarket.ru/info,10506,20268
7,https://supermarket.ru/account,10482,59975
8,https://supermarket.ru/promo_action_2068,10335,15853
9,https://supermarket.ru/other,10222,87043


In [196]:
q = '''
SELECT
    ClientID,
    groupArray(URL) as path,
    arraySlice(groupArray(URL), 1, 5) as path_lim
FROM
    (SELECT * FROM metrica_data.hits ORDER BY ClientID, EventTime)
GROUP BY ClientID
LIMIT 10
FORMAT TabSeparatedWithNames
'''

my_client.get_clickhouse_df(q)

,ClientID,path,path_lim
0,2773878518280827494,['https://supermarket.ru/promo_action_2068'],['https://supermarket.ru/promo_action_2068']
1,11346328702551058579,"['https://supermarket.ru/product_8046','https:...","['https://supermarket.ru/product_8046','https:..."
2,13106442848447404077,"['https://supermarket.ru/login','https://super...","['https://supermarket.ru/login','https://super..."
3,13041872533683226559,"['https://supermarket.ru/product_1858','https:...","['https://supermarket.ru/product_1858','https:..."
4,14051588439054376991,"['https://supermarket.ru/product_1036','https:...","['https://supermarket.ru/product_1036','https:..."
5,1880557729445743004,['https://supermarket.ru/product_2556'],['https://supermarket.ru/product_2556']
6,17287937153480005918,"['https://supermarket.ru/','https://supermarke...","['https://supermarket.ru/','https://supermarke..."
7,1717899126568995497,"['https://supermarket.ru/product_1174','https:...","['https://supermarket.ru/product_1174','https:..."
8,12735858831012285246,"['https://supermarket.ru/','https://supermarke...","['https://supermarket.ru/','https://supermarke..."
9,16757513062212456860,"['https://supermarket.ru/','https://supermarke...","['https://supermarket.ru/','https://supermarke..."


In [197]:
q = '''
SELECT
    path_lim,
    count() as users
FROM
    (SELECT
        ClientID,
        arraySlice(groupArray(URL), 1, 5) as path_lim
    FROM
        (SELECT * FROM metrica_data.hits ORDER BY ClientID, EventTime)
    GROUP BY ClientID)
GROUP BY path_lim
ORDER BY users desc
LIMIT 10
FORMAT TabSeparatedWithNames
'''

my_client.get_clickhouse_df(q)

,path_lim,users
0,['https://supermarket.ru/product_1174'],7931
1,"['https://supermarket.ru/product_1174','https:...",6827
2,['https://supermarket.ru/promo_action_2068'],6717
3,"['https://supermarket.ru/','https://supermarke...",6005
4,"['https://supermarket.ru/search','https://supe...",3918
5,['https://supermarket.ru/promo_actions'],3582
6,"['https://supermarket.ru/promo_actions','https...",3426
7,['https://supermarket.ru/promo_action_3710'],3374
8,['https://supermarket.ru/'],3304
9,['https://supermarket.ru/search'],3253


Пофильтруем повторяющиеся шаги в путях, чтобы было понятнее, в каких разделах были пользователи

In [198]:
q = '''
SELECT
    path,
    count() as users,
    arrayEnumerate(path) as indexes,
    arrayFilter(val, index -> (index = 1) or (path[index - 1] != path[index]),
        path, indexes) as path_filt
FROM
    (SELECT
        ClientID,
        groupArray(URL) as path
    FROM
        (SELECT * FROM metrica_data.hits ORDER BY ClientID, EventTime)
    GROUP BY ClientID)
GROUP BY path
ORDER BY users desc
LIMIT 10
FORMAT TabSeparatedWithNames
'''

my_client.get_clickhouse_df(q)

ValueError: Code: 241, e.displayText() = DB::Exception: Memory limit (for query) exceeded: would use 4.55 GiB (attempt to allocate chunk of 4294969752 bytes), maximum: 3.00 GiB (version 20.8.12.2 (official build))


In [202]:
q = '''
SELECT 
    path_filt,
    sum(users) as users
FROM
    (SELECT
        path,
        count() as users,
        arrayEnumerate(path) as indexes,
        arrayFilter(val, index -> (index = 1) or (path[index - 1] != path[index]),
            path, indexes) as path_filt
    FROM
        (SELECT
            ClientID,
            groupArray(URL) as path
        FROM
            (SELECT * FROM metrica_data.hits ORDER BY ClientID, EventTime)
        GROUP BY ClientID)
    GROUP BY path)
GROUP BY path_filt
ORDER BY users DESC
LIMIT 10
FORMAT TabSeparatedWithNames
'''

my_client.get_clickhouse_df(q)

ValueError: Code: 241, e.displayText() = DB::Exception: Memory limit (for query) exceeded: would use 4.55 GiB (attempt to allocate chunk of 4294969752 bytes), maximum: 3.00 GiB (version 20.8.12.2 (official build))


## Расчет funnels

#### Подход в лоб

In [215]:
q = '''
SELECT
    ClientID,
    max(
        URL in ('https://supermarket.ru/catalog', 'https://supermarket.ru/search')) as step1_achieved,
    sequenceMatch('(?1).*(?2)')(EventTime,
        URL in ('https://supermarket.ru/catalog', 'https://supermarket.ru/search'),
        URL = 'https://supermarket.ru/basket') as step2_achieved,
    sequenceMatch('(?1).*(?2).*(?3)')(EventTime, 
        URL in ('https://supermarket.ru/catalog', 'https://supermarket.ru/search'),
        URL = 'https://supermarket.ru/basket',
        URL = 'https://supermarket.ru/order_step_1') as step3_achieved,
    sequenceMatch('(?1).*(?2).*(?3).*(?4)')(EventTime, 
        URL in ('https://supermarket.ru/catalog', 'https://supermarket.ru/search'),
        URL = 'https://supermarket.ru/basket',
        URL = 'https://supermarket.ru/order_step_1', 
        URL = 'https://supermarket.ru/order_step_2') as step4_achieved,
    sequenceMatch('(?1).*(?2).*(?3).*(?4).*(?5)')(EventTime, 
        URL in ('https://supermarket.ru/catalog', 'https://supermarket.ru/search'),
        URL = 'https://supermarket.ru/basket',
        URL = 'https://supermarket.ru/order_step_1', 
        URL = 'https://supermarket.ru/order_step_2', 
        URL = 'https://supermarket.ru/order_step_3') as step5_achieved
FROM metrica_data.hits
GROUP BY ClientID
LIMIT 10
FORMAT TabSeparatedWithNames
'''

my_client.get_clickhouse_df(q)

,ClientID,step1_achieved,step2_achieved,step3_achieved,step4_achieved,step5_achieved
0,2773878518280827494,0,0,0,0,0
1,11346328702551058579,0,0,0,0,0
2,13106442848447404077,0,0,0,0,0
3,13041872533683226559,0,0,0,0,0
4,14051588439054376991,0,0,0,0,0
5,1880557729445743004,0,0,0,0,0
6,17287937153480005918,0,0,0,0,0
7,1717899126568995497,0,0,0,0,0
8,12735858831012285246,1,0,0,0,0
9,16757513062212456860,0,0,0,0,0


In [219]:
q = '''
SELECT
    sum(step1_achieved) as step1,
    sum(step2_achieved) as step2,
    sum(step3_achieved) as step3,
    sum(step4_achieved) as step4,
    sum(step5_achieved) as step5
    --round(100.*step2/step1, 2) as step2_to_1,
    --round(100.*step3/step1, 2) as step3_to_1
FROM
    (SELECT
        ClientID,
        max(
            URL in ('https://supermarket.ru/catalog', 'https://supermarket.ru/search')) as step1_achieved,
        sequenceMatch('(?1).*(?2)')(EventTime,
            URL in ('https://supermarket.ru/catalog', 'https://supermarket.ru/search'),
            URL = 'https://supermarket.ru/basket') as step2_achieved,
        sequenceMatch('(?1).*(?2).*(?3)')(EventTime, 
            URL in ('https://supermarket.ru/catalog', 'https://supermarket.ru/search'),
            URL = 'https://supermarket.ru/basket',
            URL = 'https://supermarket.ru/order_step_1') as step3_achieved,
        sequenceMatch('(?1).*(?2).*(?3).*(?4)')(EventTime, 
            URL in ('https://supermarket.ru/catalog', 'https://supermarket.ru/search'),
            URL = 'https://supermarket.ru/basket',
            URL = 'https://supermarket.ru/order_step_1', 
            URL = 'https://supermarket.ru/order_step_2') as step4_achieved,
        sequenceMatch('(?1).*(?2).*(?3).*(?4).*(?5)')(EventTime, 
            URL in ('https://supermarket.ru/catalog', 'https://supermarket.ru/search'),
            URL = 'https://supermarket.ru/basket',
            URL = 'https://supermarket.ru/order_step_1', 
            URL = 'https://supermarket.ru/order_step_2', 
            URL = 'https://supermarket.ru/order_step_3') as step5_achieved
    FROM metrica_data.hits
    GROUP BY ClientID)
FORMAT TabSeparatedWithNames
'''

fdf = my_client.get_clickhouse_df(q)

In [220]:
fdf

,step1,step2,step3,step4,step5
0,37419,9746,7381,6566,6404


Визуализируем

In [207]:
from plotly.offline import init_notebook_mode, iplot
import plotly
import plotly.graph_objs as go

init_notebook_mode(connected=True)

In [222]:
colors = colors = ['#d54936', '#faca34', '#437cba', '#8bc34a', '#795548', '#309688', '#000000', '#40bcd4', '#9e9e9e', '#3ca9f4']
phases = ['Search / Catalog', 'Basket', 'Order step 1', 'Order step 2', 'Order finished']
values = fdf.iloc[0].values

In [223]:
def plot_funnel(phases, values):
    n_phase = len(phases)
    plot_width = 500.

    # height of a section and difference between sections 
    section_h = 100
    section_d = 10

    # multiplication factor to calculate the width of other sections
    unit_width = plot_width / max(values)

    # width of each funnel section relative to the plot width
    phase_w = [int(value * unit_width) for value in values]
    print (phase_w)

    # plot height based on the number of sections and the gap in between them
    height = section_h * n_phase + section_d * (n_phase - 1)
    
    # list containing all the plot shapes
    shapes = []

    # list containing the Y-axis location for each section's name and value text
    label_y = []

    for i in range(n_phase):
            if (i == n_phase-1):
                    points = [phase_w[i] / 2, height, phase_w[i] / 2, height - section_h]
            else:
                    points = [phase_w[i] / 2, height, phase_w[i+1] / 2, height - section_h]

            path = 'M {0} {1} L {2} {3} L -{2} {3} L -{0} {1} Z'.format(*points)

            shape = {
                    'type': 'path',
                    'path': path,
                    'fillcolor': colors[i],
                    'line': {
                        'width': 1,
                        'color': colors[i]
                    }
            }
            shapes.append(shape)

            # Y-axis location for this section's details (text)
            label_y.append(height - (section_h / 2))

            height = height - (section_h + section_d)

    # For phase names
    label_trace = go.Scatter(
        x=[-350]*n_phase,
        y=label_y,
        mode='text',
        text=phases,
        textfont=dict(
            color='rgb(40,40,40)',
            size=15
        )
    )

    # For phase values
    value_trace = go.Scatter(
        x=[350]*n_phase,
        y=label_y,
        mode='text',
        text=values,
        textfont=dict(
            color='rgb(40,40,40)',
            size=15
        )
    )

    data = [label_trace, value_trace]

    layout = go.Layout(
        title="<b>Funnel Chart</b>",
        titlefont=dict(
            size=20,
            color='rgb(0,0,0)'
        ),
        shapes=shapes,
        height=560,
        width=800,
        showlegend=False,
        paper_bgcolor='rgba(255,255,255,1)',
        plot_bgcolor='rgba(255,255,255,1)',
        xaxis=dict(
            showticklabels=False,
            zeroline=False,
            showgrid=False,
            range=[-450, 450]
        ),
        yaxis=dict(
            showticklabels=False,
            zeroline=False,
            showgrid=False
        )
    )

    fig = go.Figure(data=data, layout=layout)
    iplot(fig, show_link=False)

In [224]:
plot_funnel(phases, values)

[500, 130, 98, 87, 85]


### Пойдем другим путем

In [226]:
q = '''
SELECT
    ClientID,
    groupArray(cast(EventTime as UInt64)) as times,
    groupArray(URL) as urls,
    arrayFilter(time, url -> url in ('https://supermarket.ru/catalog', 'https://supermarket.ru/search'), times, urls)[1] as step1_time
FROM
    (SELECT * FROM metrica_data.hits ORDER BY ClientID, EventTime)
GROUP BY ClientID
LIMIT 5
FORMAT TabSeparatedWithNames
'''

my_client.get_clickhouse_df(q)

,ClientID,times,urls,step1_time
0,2773878518280827494,[1593682943],['https://supermarket.ru/promo_action_2068'],0
1,11346328702551058579,"[1594907928,1594907943,1594907962,1594907981,1...","['https://supermarket.ru/product_8046','https:...",0
2,13106442848447404077,"[1600246925,1600246953,1600247004,1600247041,1...","['https://supermarket.ru/login','https://super...",0
3,13041872533683226559,"[1596262807,1596262822]","['https://supermarket.ru/product_1858','https:...",0
4,14051588439054376991,"[1594202050,1594202071]","['https://supermarket.ru/product_1036','https:...",0


In [227]:
q = '''
SELECT
    ClientID,
    groupArray(cast(EventTime as UInt64)) as times,
    groupArray(URL) as urls,
    arrayFilter(time, url -> url in ('https://supermarket.ru/catalog', 'https://supermarket.ru/search'), times, urls)[1] as step1_time,
    arrayFilter(time, url -> url = 'https://supermarket.ru/basket' and step1_time != 0 and time >= step1_time, times, urls)[1] as step2_time
FROM
    (SELECT * FROM metrica_data.hits ORDER BY ClientID, EventTime)
GROUP BY ClientID
LIMIT 5
FORMAT TabSeparatedWithNames
'''

my_client.get_clickhouse_df(q)

,ClientID,times,urls,step1_time,step2_time
0,2773878518280827494,[1593682943],['https://supermarket.ru/promo_action_2068'],0,0
1,11346328702551058579,"[1594907928,1594907943,1594907962,1594907981,1...","['https://supermarket.ru/product_8046','https:...",0,0
2,13106442848447404077,"[1600246925,1600246953,1600247004,1600247041,1...","['https://supermarket.ru/login','https://super...",0,0
3,13041872533683226559,"[1596262807,1596262822]","['https://supermarket.ru/product_1858','https:...",0,0
4,14051588439054376991,"[1594202050,1594202071]","['https://supermarket.ru/product_1036','https:...",0,0


In [229]:
q = '''
SELECT
    countIf(step1_time != 0) as step1,
    countIf(step2_time != 0) as step2,
    countIf(step3_time != 0) as step3,
    countIf(step4_time != 0) as step4,
    countIf(step5_time != 0) as step5
FROM
    (SELECT
        ClientID,
        groupArray(cast(EventTime as UInt64)) as times,
        groupArray(URL) as urls,
        arrayFilter(time, url -> url in ('https://supermarket.ru/catalog', 'https://supermarket.ru/search'), times, urls)[1] as step1_time,
        arrayFilter(time, url -> url = 'https://supermarket.ru/basket' and step1_time != 0 and time >= step1_time, times, urls)[1] as step2_time,
        arrayFilter(time, url -> url = 'https://supermarket.ru/order_step_1' and step2_time != 0 and time >= step2_time, times, urls)[1] as step3_time,
        arrayFilter(time, url -> url = 'https://supermarket.ru/order_step_2' and step3_time != 0 and time >= step3_time, times, urls)[1] as step4_time,
        arrayFilter(time, url -> url = 'https://supermarket.ru/order_step_3' and step4_time != 0 and time >= step4_time, times, urls)[1] as step5_time
    FROM
        (SELECT * FROM metrica_data.hits ORDER BY ClientID, EventTime)
    GROUP BY ClientID)
FORMAT TabSeparatedWithNames
'''

fdf = my_client.get_clickhouse_df(q)

In [230]:
phases = ['Search / Catalog', 'Basket', 'Order step 1', 'Order step 2', 'Order finished']
values = fdf.iloc[0].values

plot_funnel(phases, values)

[500, 130, 98, 87, 85]


Можем также посчитать среднее время достижения шагов

In [231]:
q = '''
SELECT
    medianIf(step2_time - step1_time, step2_time != 0) as step2_from_step1_mean_time,
    medianIf(step3_time - step2_time, step3_time != 0) as step3_from_step2_mean_time,
    medianIf(step4_time - step3_time, step4_time != 0) as step4_from_step3_mean_time,
    medianIf(step5_time - step4_time, step5_time != 0) as step5_from_step4_mean_time
FROM
    (SELECT
        ClientID,
        groupArray(cast(EventTime as UInt64)) as times,
        groupArray(URL) as urls,
        arrayFilter(time, url -> url in ('https://supermarket.ru/catalog', 'https://supermarket.ru/search'), times, urls)[1] as step1_time,
        arrayFilter(time, url -> url = 'https://supermarket.ru/basket' and step1_time != 0 and time >= step1_time, times, urls)[1] as step2_time,
        arrayFilter(time, url -> url = 'https://supermarket.ru/order_step_1' and step2_time != 0 and time >= step2_time, times, urls)[1] as step3_time,
        arrayFilter(time, url -> url = 'https://supermarket.ru/order_step_2' and step3_time != 0 and time >= step3_time, times, urls)[1] as step4_time,
        arrayFilter(time, url -> url = 'https://supermarket.ru/order_step_3' and step4_time != 0 and time >= step4_time, times, urls)[1] as step5_time
    FROM
        (SELECT * FROM metrica_data.hits ORDER BY ClientID, EventTime)
    GROUP BY ClientID)
FORMAT TabSeparatedWithNames
'''

my_client.get_clickhouse_df(q)

,step2_from_step1_mean_time,step3_from_step2_mean_time,step4_from_step3_mean_time,step5_from_step4_mean_time
0,1118,675,81,53


# Воронки по браузеру

In [232]:
q = '''
SELECT
    Browser,
    countIf(step1_time != 0) as step1,
    countIf(step2_time != 0) as step2,
    countIf(step3_time != 0) as step3,
    countIf(step4_time != 0) as step4,
    countIf(step5_time != 0) as step5
FROM
(
    SELECT
        Browser,
        ClientID,
        groupArray(cast(EventTime as UInt64)) as times,
        groupArray(URL) as urls,
        arrayFilter(time, url -> url in ('https://supermarket.ru/catalog', 'https://supermarket.ru/search'), times, urls)[1] as step1_time,
        arrayFilter(time, url -> url = 'https://supermarket.ru/basket' and step1_time != 0 and time >= step1_time, times, urls)[1] as step2_time,
        arrayFilter(time, url -> url = 'https://supermarket.ru/order_step_1' and step2_time != 0 and time >= step2_time, times, urls)[1] as step3_time,
        arrayFilter(time, url -> url = 'https://supermarket.ru/order_step_2' and step3_time != 0 and time >= step3_time, times, urls)[1] as step4_time,
        arrayFilter(time, url -> url = 'https://supermarket.ru/order_step_3' and step4_time != 0 and time >= step4_time, times, urls)[1] as step5_time
    FROM
        (SELECT * FROM metrica_data.hits ORDER BY ClientID, EventTime)
    GROUP BY Browser, ClientID
)
GROUP BY Browser
FORMAT TabSeparatedWithNames
'''

fdf = my_client.get_clickhouse_df(q)

In [235]:
fdf.sort_values('step1', ascending=False).head()

,Browser,step1,step2,step3,step4,step5
27,chrome,12366,3880,3110,2812,2744
47,safari_mobile,4678,902,581,515,502
38,chromemobile,4220,433,197,131,129
46,yandex_browser,4146,1241,956,842,814
10,firefox,2776,1002,797,727,713


In [237]:
get_clickhouse_data('drop table if exists metrica_data.funnels_by_bro')

q = '''
create table metrica_data.funnels_by_bro Engine = Log as
SELECT
    Browser,
    countIf(step1_time != 0) as step1,
    countIf(step2_time != 0) as step2,
    countIf(step3_time != 0) as step3,
    countIf(step4_time != 0) as step4,
    countIf(step5_time != 0) as step5
FROM
(
    SELECT
        Browser,
        ClientID,
        groupArray(cast(EventTime as UInt64)) as times,
        groupArray(URL) as urls,
        arrayFilter(time, url -> url in ('https://supermarket.ru/catalog', 'https://supermarket.ru/search'), times, urls)[1] as step1_time,
        arrayFilter(time, url -> url = 'https://supermarket.ru/basket' and step1_time != 0 and time >= step1_time, times, urls)[1] as step2_time,
        arrayFilter(time, url -> url = 'https://supermarket.ru/order_step_1' and step2_time != 0 and time >= step2_time, times, urls)[1] as step3_time,
        arrayFilter(time, url -> url = 'https://supermarket.ru/order_step_2' and step3_time != 0 and time >= step3_time, times, urls)[1] as step4_time,
        arrayFilter(time, url -> url = 'https://supermarket.ru/order_step_3' and step4_time != 0 and time >= step4_time, times, urls)[1] as step5_time
    FROM
        (SELECT * FROM metrica_data.hits ORDER BY ClientID, EventTime)
    GROUP BY Browser, ClientID
)
GROUP BY Browser
'''

my_client.get_clickhouse_data(q)

''